In [332]:
#@title Import
%%capture
import os, json
import numpy as np
import pandas as pd

from google.colab import drive

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from skimage import io
from datetime import datetime, timedelta

In [150]:
#@title Conntect Google Drive
%%capture
drive.mount("/content/drive")

In [151]:
path = "/content/drive/MyDrive/(07) Colab/20231218_tp_video/images/"
files = os.listdir(path)
files = [path + i for i in files]

In [ ]:
# #@title Get Model
# %%capture
# from transformers import pipeline
# od_model = pipeline("object-detection", "facebook/detr-resnet-50")

In [152]:
# %%capture
# od_result = od_model(files, device = 0)

# with open("/content/drive/MyDrive/(07) Colab/20231218_tp_video/od_result.json", "w") as file:
#   json.dump(od_result, file)

with open("/content/drive/MyDrive/(07) Colab/20231218_tp_video/od_result.json", "r") as file:
  od_result = json.load(file)

In [385]:
label = []

for i in od_result:
  for j in i:
    label.extend([j["label"]])

label = sorted(set(label))

In [451]:
labels = [[j["label"] for j in i] for i in od_result]
labels = { i: [sum([i == k for k in j]) for j in labels] for i in ["bicycle", "bus", "car", "motorcycle", "person", "truck"] }

labels["path"] = files

data = pd.DataFrame(labels)

data["datetime"] = pd.to_datetime(data["path"].apply(lambda x: x.split("/")[-1].split(".")[0]))
data["datehour"] = data["datetime"].map(lambda x: x.round(freq = "1H"))
data["datetime5s"] = data["datetime"].map(lambda x: x.round(freq = "5S"))
data["datetime5t"] = data["datetime"].map(lambda x: x.round(freq = "5T"))

data = data.query("datetime > '2023-12-19 03:00:00'").sort_values(["datetime"])

In [452]:
color_map = {"bicycle": "blue", "bus": "green", "car": "red", "motorcycle": "orange", "person": "purple", "truck": "brown"}

In [476]:
hourly_summary = data.\
  groupby(["datehour", "datetime5s"], as_index = False).\
  agg(bus = ("bus", "mean"),
      car = ("car", "mean"),
      motorcycle = ("motorcycle", "mean"),
      truck = ("truck", "mean"),
      bicycle = ("bicycle", "mean"),
      person = ("person", "mean")).\
  groupby(["datehour"], as_index = False).\
  agg(bus = ("bus", "sum"),
      car = ("car", "sum"),
      motorcycle = ("motorcycle", "sum"),
      truck = ("truck", "sum"),
      bicycle = ("bicycle", "sum"),
      person = ("person", "sum")).\
  sort_values(["datehour"])

hourly_summary = pd.melt(hourly_summary, id_vars=["datehour"], value_vars=["bicycle", "bus", "car", "motorcycle", "person", "truck"])
hourly_summary.value = hourly_summary.value.round()
hourly_summary.datehour = hourly_summary.datehour.apply(lambda x: x.tz_localize("UTC").tz_convert("Asia/Taipei"))

px.bar(hourly_summary, x = "value", y = "datehour", color = "variable",
       text_auto=True, color_discrete_map = color_map,
       width = 900, height = 600).\
       update_xaxes(title = "Count", tickformat = ",").\
       update_yaxes(title = "Hour", tickformat = "%H:%M").\
       update_layout(template="plotly_white",
                     title = "Hourly Traffic Flow at the Intersection",
                     legend=dict(title = "", yanchor="top", y=0.99, xanchor="left", x=0.80 ))

In [478]:
minutes_summary = data.\
  groupby(["datetime5t", "datetime5s"], as_index = False).\
  agg(bus = ("bus", "mean"),
      car = ("car", "mean"),
      motorcycle = ("motorcycle", "mean"),
      truck = ("truck", "mean"),
      bicycle = ("bicycle", "mean"),
      person = ("person", "mean"),
      path = ("path", "first")).\
  groupby(["datetime5t"], as_index = False).\
  agg(bus = ("bus", "sum"),
      car = ("car", "sum"),
      motorcycle = ("motorcycle", "sum"),
      truck = ("truck", "sum"),
      bicycle = ("bicycle", "sum"),
      person = ("person", "sum"),
      path = ("path", "first")).\
  sort_values(["datetime5t"])

minutes_summary.datetime5t = minutes_summary.datetime5t.apply(lambda x: x.tz_localize("UTC").tz_convert("Asia/Taipei"))

minutes_summary

,datetime5t,bus,car,motorcycle,truck,bicycle,person,path
0,2023-12-19 11:00:00+08:00,2.80,11.000000,24.533333,2.800000,0.000000,32.266667,/content/drive/MyDrive/(07) Colab/20231218_tp_...
1,2023-12-19 11:05:00+08:00,1.75,4.800000,5.950000,0.750000,1.750000,27.000000,/content/drive/MyDrive/(07) Colab/20231218_tp_...
2,2023-12-19 11:10:00+08:00,2.00,45.950000,54.016667,10.000000,0.333333,92.716667,/content/drive/MyDrive/(07) Colab/20231218_tp_...
3,2023-12-19 11:15:00+08:00,1.00,22.150000,71.950000,0.250000,0.500000,79.300000,/content/drive/MyDrive/(07) Colab/20231218_tp_...
4,2023-12-19 11:20:00+08:00,0.00,50.966667,105.933333,13.633333,0.000000,98.233333,/content/drive/MyDrive/(07) Colab/20231218_tp_...
...,...,...,...,...,...,...,...,...
129,2023-12-19 21:45:00+08:00,0.00,54.300000,24.733333,0.000000,0.000000,66.500000,/content/drive/MyDrive/(07) Colab/20231218_tp_...
130,2023-12-19 21:50:00+08:00,0.00,44.333333,56.300000,0.000000,0.000000,92.050000,/content/drive/MyDrive/(07) Colab/20231218_tp_...
131,2023-12-19 21:55:00+08:00,0.40,65.700000,19.600000,0.800000,0.000000,38.600000,/content/drive/MyDrive/(07) Colab/20231218_tp_...
132,2023-12-19 22:00:00+08:00,0.00,26.933333,15.666667,0.000000,0.000000,24.066667,/content/drive/MyDrive/(07) Colab/20231218_tp_...


In [472]:
start = 2
obs = len(minutes_summary)

minutes_summary_animate = pd.DataFrame()

for i in np.arange(start, obs):
    minutes_summary_animate = pd.concat([minutes_summary_animate, minutes_summary.head(i).copy().assign(ix = i)])


minutes_summary_animate

,datetime5t,bus,car,motorcycle,truck,bicycle,person,ix
0,2023-12-19 11:00:00+08:00,2.80,11.000000,24.533333,2.80,0.000000,32.266667,2
1,2023-12-19 11:05:00+08:00,1.75,4.800000,5.950000,0.75,1.750000,27.000000,2
0,2023-12-19 11:00:00+08:00,2.80,11.000000,24.533333,2.80,0.000000,32.266667,3
1,2023-12-19 11:05:00+08:00,1.75,4.800000,5.950000,0.75,1.750000,27.000000,3
2,2023-12-19 11:10:00+08:00,2.00,45.950000,54.016667,10.00,0.333333,92.716667,3
...,...,...,...,...,...,...,...,...
128,2023-12-19 21:40:00+08:00,0.60,63.400000,23.650000,0.40,0.533333,56.333333,133
129,2023-12-19 21:45:00+08:00,0.00,54.300000,24.733333,0.00,0.000000,66.500000,133
130,2023-12-19 21:50:00+08:00,0.00,44.333333,56.300000,0.00,0.000000,92.050000,133
131,2023-12-19 21:55:00+08:00,0.40,65.700000,19.600000,0.80,0.000000,38.600000,133


In [495]:
fig_trend = px.line(minutes_summary_animate,
                    x = "datetime5t",
                    y = ["bicycle", "bus", "car", "motorcycle", "person", "truck"],
                    color_discrete_map = color_map, animation_frame = "ix",
                    width = 600, height = 400).\
        update_xaxes(title = "Time", tickformat = "%H:%M").\
        update_yaxes(title = "").\
        update_layout(template="plotly_white",
                      title = "Traffic Flow Trend Every Five Minutes",
                      legend=dict(title = "", yanchor="top", y=1, xanchor="left", x=0.01, bgcolor = "rgba(0,0,0,0)"))

fig_trend.layout["updatemenus"][0]["buttons"][0]["args"][1]["frame"]["duration"] = 100
fig_trend.layout["sliders"][0]["visible"] = False
fig_trend.layout.updatemenus[0].buttons[0]['args'][1]['frame']['redraw'] = True

#fig_trend.show()

In [487]:
images = np.array([io.imread(i) for i in minutes_summary.path[:obs]])
boxes = [od_result[j] for j in [files.index(i) for i in minutes_summary.path[:obs].tolist()]]

In [492]:
np.save("/content/drive/MyDrive/(07) Colab/20231218_tp_video/animation_images.npy", images)

with open("/content/drive/MyDrive/(07) Colab/20231218_tp_video/animation_boxes.json", "w") as f:
    json.dump(boxes, f)

In [493]:
images2 = np.load("/content/drive/MyDrive/(07) Colab/20231218_tp_video/animation_images.npy")

In [494]:
fig_image = px.imshow(images, animation_frame=0, binary_string=True, width=600, height=400)

fig_image.update_xaxes(showticklabels=False)
fig_image.update_yaxes(showticklabels=False)

for i in range(len(fig_image.frames)):

  shapes = []

  for j in boxes[i]:

    if sum([k == j["label"] for k in ["bus", "car", "motorcycle", "truck"]])>0:

      xmin, ymin, xmax, ymax = j["box"].values()

      shapes.append(go.layout.Shape(type = "rect", x0 = xmin, y0 = ymin, x1 = xmax, y1=ymax, line = dict(color = color_map[j["label"]], width = 1.5)))

  fig_image.frames[i].layout.shapes = shapes

fig_image.layout["updatemenus"][0]["buttons"][0]["args"][1]["frame"]["duration"] = 100
fig_image.layout["sliders"][0]["visible"] = True

# fig_image.show()